# Конструирование суррогатной модели

In [1]:
import numpy as np
import matplotlib.pyplot as plt

Основная задача этой главы - изучить отображение $y = f(x)$ , производящееся в так называемом "чёрном ящике", который скрывает физику этого отображения, преобразующего вектор __x__ в скалярную величину $y$ . Этот черный ящик может принимать форму физического или компьютерного эксперимента, например, кода, реализующего метод конечных элементов, который вычисляет максимальное напряжение (f) для заданных размеров изделия (__x__). Общий метод решения данной проблемы состоит следующем: сначала собираются выходные значения $y^{(1)},y^{(2)},...,y^{(n)}$, каждое из которых является результатом действия отображения $f$ на входные данные $x^{(1)},x^{(2)},...,x^{(n)}$, а затем (на основе наблюдений) ищется функция $\hat{f}(x)$, которая описывает отображение чёрного ящика $f$ наилучшим образом.  
В этой главе мы обсудим основы и некоторые технические детали ряда конкретных типов суррогатных моделей, способных решить поставленную задачу. Однако начнём мы с обсуждения ключевых этапов процесса построения суррогатной модели.

## Процесс Моделирования

### Первая стадия: Подготовка данных и выбор Подхода к моделированию

В разделе 1 рассматриваются два предварительных шага этого этапа. Первый - это идентификация: с помощью небольшого числа наблюдений определить те параметры входных данных, которые оказывают существенное влияние на $f$; то есть происходит определение кратчайшего вектора проектной переменной $x = \{x_1, x_2,..., x_k\}^T$, который, в рассматриваемых диапозонах всех своих компонент может сильно влиять на поведение чёрного ящика. Если бы чёрный ящик был электронным устройством, оснащенным большим набором элементов управления, то этот шаг сводился бы к выявлению $k$ элементов управления, которые при изменении влияют на его поведение. Эта операция осложнена возможными взаимодействиями между элементами управления. На этом этапе также необходимо установить диапазоны различных проектных переменных.  

Второй шаг - сбор $n$ штук векторов длины $k$ в список $X = \{ x^{(1)}, x^{(2)},...,x^{(n)} \}^T$, таким образом, чтобы этот набор максимально полно представлял собой пространство проектирования. Основная проблема в том, что n часто невелико, т.к. оно ограничено вычислительной и временной стоимостью, приходящейся на каждое наблюдение $f(x_i)$.  
Возможно, здесь стоит повторить, что на этом шаге неплохо масштабировать x в единичный куб $[0,1]^k$, что может упростить некоторые последующие математические вычисления и избавить нас от множества проблем многомерного масштабирования.  

Поскольку в главе 1 изложен ряд методов для выполнения вышеуказанного, здесь мы рассмотрим следующую фазу процесса - фактическую попытку изучения $f$ с помощью пар данных $\{(x^{(1)},y^{(1)}),(x^{(2)},y^{(2)}), ...(x^{(n)},y^{(n)})\}$. Этот так называемый _процесс обучения под наблюдением или на основе примеров_, по сути, представляет собой поиск в пространстве всех возможных функций $\hat{f}$, которые воспроизводили бы наблюдения $f$.  

Это пространство бесконечно. В конце концов, можно нарисовать любое количество (гипер)поверхностей, прохождящих в определенном диапазоне (с учетом экспериментальной ошибки) известных наблюдений. Однако подавляющее большинство из них будет очень плохо обобщаться; то есть они будут практически бесполезны для прогнозирования ответов на новых данных, в чем и заключается цель данной задачи.

Рассмотрим несколько экстремальный пример функции "иголка(иголки) в стоге сена":


$$ \hat{f} = \begin{cases}
   y^{(1)} &\text{if $x = x^{(1)}$}\\
   y^{(2)} &\text{if $x = x^{(2)}$}\\
   ...\\
   y^{(n)} &\text{if $x = x^{(n)}$}\\
   0 &\text{иначе}\\ 
 \end{cases} $$

Очевидно, что, хотя все обучающие данные могут быть воспроизведены с помощью этого предиктора, но есть, по крайней мере, в отношении большинства инженерных функций, что-то сильно противоречащее интуиции и тревожащее в том, что он предсказывает 0 везде, кроме обучающих точек. Конечно, есть небольшая вероятность того, что функция действительно выглядит как (1), и по какой-то необычайной случайности нам довелось исследовать её именно там, где находятся все иглы, но это маловероятно.

Можно было бы придумать множество других, возможно, менее надуманных примеров, которые также кажутся какими-то неестественными и, более того, плохо обобщаются. В конечном счете все это говорит о том, что нам нужны какие-то систематические инструменты фильтрации таких бессмысленных предсказателей. Некоторые ученые занимают здесь Байесовскую позицию, например, философия, отстаиваемая Расмуссеном и Уильямсом (2006), заключается в том, чтобы "заранее дать вероятность каждой возможной функции, причём более высокие вероятности дать функциям, которые мы считаем более вероятными, например, потому что они более гладкие, чем другие функции".  

Подход, который мы будем использовать в дальнейшем, заключается в том, чтобы заранее опредлить структуру $\hat{f}$ (например, полином третей степени), встроить её в алгоритм моделирования и выполнить поиск по пространству её параметров для настройки приближения к наблюдениям. Например, рассмотрим одну из простейших возможных моделей: $\hat{f}(x,w) = w^T * x + v$. Изучение f с помощью этой модели подразумевает, что мы определились с ее структурой – это будет гиперплоскость – и процесс подгонки модели состоит в нахождении $k + 1$ параметров (вектор наклона w и перехват v), для которых значение выражения $w^T * x + v$ наилучшим образом соответствует данным (это будет выполнено на второй стадии).  

Все вышесказанное часто дополнительно осложняется наличием шума в наблюдаемых откликах (будем считать, что проектные вектора \textbf{x} никоим образом не повреждены). Мы обсуждали природу этого шума в начале главы 1. Здесь мы концентрируемся на изучении таких данных, для которых иногда присутсвует вероятность ___перенасыщения___.
Перенасыщение происходит, когда модель в некотором смысле слишком гибкая и слишком точно соответствует обучающим данным, то есть соответствует шуму, а также фактическому базовому поведению, которое мы пытаемся смоделировать. На второй стадии процесса суррогатного строительства эта проблема _управления сложностью_ решается с помощью процесса оценки параметров модели фиксированной конструкции, но здесь, на этапе выбора типа модели, также требуется некоторое предвидение в этом направлении.  

Обычно этот шаг связан с физическими соображениями; то есть выбор метода моделирования зависит от наших ожиданий того, как может выглядеть лежащий в основе ответ. Например, если у нас есть некоторые наблюдения за напряжением в упруго деформированном твердом теле в ответ на небольшие деформации, имеет смысл моделировать напряжение с помощью простого линейного приближения.  
Если такое понимание физики недоступно и мы не сможем учесть, скажем, линейность данных на данном этапе, мы в конечном итоге примем сложную, чрезмерно гибкую модель. Это не будет концом света (в конце концов, этап оценки параметров, как мы надеемся, "линеаризует" аппроксимацию путем соответствующего выбора параметров, управляющих ее формой), но мы упустим возможность получить алгебраически простую, надежную модель.  

И наоборот, если мы ошибочно предположим, что данные получены, скажем, из базового квадратичного процесса, а на самом деле истинная функция $f$ имеет множество пиков и впадин, на этапе оценки параметров мы не сможем компенсировать неудачно выбранную модель. Квадратичная просто будет слишком жесткой, чтобы соответствовать мультимодальной функции, каковы бы ни были ее параметры.

### Вторая стадия: Оценка параметров и обучение

Давайте предположим, что на первой стадии мы определили $k$ проектных переменных, получили набор обучающих данных и выбрали общую структуру модели $\hat{f}(x, w)$, где точная форма модели определяется набором параметров $w$. Теперь мы сталкиваемся с проблемой оценки параметров: как нам выбрать такие $w$, чтобы модель наилучшим образом соответствовала данным? Существует много критериев оценки, но здесь мы обсудим два.

#### Метод Максимального Правдоподобия

Если даны набор параметров $w$ и сама модель $\hat{f} (x, w)$, то мы можем вычислить вероятность того, что набор данных $\{(x^{(1)},y^{(1)} \pm \epsilon),(x^{(2)},y^{(2)} \pm \epsilon),...,(x^{(n)},y^{(n)} \pm \epsilon)\}$ будет получен из $\hat{f}$ (где $\epsilon$ - некоторая небольшая постоянная погрешность вокруг каждой точки). Мы можем предположить, что ошибки независимо случайным образом распределены в соответствии с нормальным распределением со стандартным отклонением $\sigma$, тогда вероятность получения данного набора данных равна  

$$P = \frac{1}{(2\pi \sigma^2)^{n/2}} \prod\limits_{i=1}^n \Bigr\{ exp \Bigr[ - \frac{1}{2} \Bigr( \frac{y^{(i)} - \hat{f}(x,w)} {\sigma}\Bigl)^2 \Bigl] \epsilon \Bigl\}$$

В этом и заключается метод максимального правдоподобия: мы предполагаем, что результаты, полученные нашей выборкой, наиболее вероятны (являются мат.ожиданием). Тогда нам нужно максимизировать значение этого выражения. Для упрощения вычислений - прологорифмируем и найдём минумум выражения с обратным знаком, приэтом избавимся от незначимых констант:  

$$\min_w \sum\limits_{i=1}^n \frac{\Bigr[y^{(i)} - \hat{f}(x,w)\Bigl]^2}{2 \sigma^2} - n \ln \epsilon$$

Заметим, что если мы предполагаем $\sigma$ и $\epsilon$ постоянными, то выражение (3) упрощается и принимает хорошо знакомый вид _метода наименьших квадратов_ :  

$$\min_w \sum\limits_{i=1}^n \Bigr[y^{(i)} - \hat{f}(x,w)\Bigl]^2$$

#### Кросс-валидация (скользящий контроль или метод перекрёстной проверки)

Перекрёстная проверка состоит из нескольких этапов: 
1. разбиение данных (случайным образом) на $q$ примерно равных по объёму подмножеств; 
2. удаление одного из этих подмножеств; 
3. обучнеие модели на оставшихся $q − 1$ подмножествах. 
4. Вычисление функции потерь $L$, которая измеряет ошибку между полученным на этапе 3) предиктором и точками в подмножестве, которое мы удалили на этапе 2).

Затем возвращаемся к этапу 2) (вернув удалённое подмножество) и удаляем другое ещё не тронутое подмножество. Снова вычисляем функцию потерь $L$ и суммируем за все $q$ итераций.  

Более формально, если отображение $\zeta$ : $\{1,....n\} \rightarrow \{1,...,q\}$ описывает распределение $n$ обучающих точек в одно из $q$ подмножеств, а $\hat{f}^{-\zeta(i)}(x)$ - значение предиктора (в точке $x$), полученное путем удаления подмножества $\zeta(i)$ (т.е. подмножества, которому соответствует наблюдение $i$), то мера перекрёстной проверки, которую мы используем здесь в качестве оценки ошибки прогнозирования, равна  

$$\varepsilon_{cv}(w) = \frac{1}{n} \sum\limits_{i=1}^n L \Bigr[y^{(i)}, \hat{f}^{-\zeta(i)}(x^{(i)},w)  \Bigl]$$


Функцию потерь $L$ можно ввести, как квадрат ошибки, тогда можно переписать уравнение (5) так:  

$$\varepsilon_{cv}(w) = \frac{1}{n} \sum\limits_{i=1}^n L \Bigr[y^{(i)} - \hat{f}^{-\zeta(i)}(x^{(i)},w)  \Bigl]^2$$

В какой степени уравнение (6) является объективной оценкой истинной ошибки прогнозирования, зависит от выбора $q$. Можно показать, что если $q = n$, то $\varepsilon_{cv}$ является почти точной оценкой истинной ошибки. Однако дисперсия этой _меры исключения_ может быть очень высокой из-за того, что $n$ подмножеств очень похожи друг на друга. Хасти и др. (2001) рекомендуют компромиссные значения $q = 5$ или $q = 10$. С практической точки зрения, использование меньшего количества подмножеств дает дополнительный бонус в виде снижения вычислительных затрат на процесс перекрестной проверки за счет сокращения количества моделей, которые необходимо подставлять.

### Третья стадия: Тестирование модели